In [ ]:
pip install MySQL-python

In [1]:
import pandas as pd
import mysql.connector

In [3]:
# Make sure to put this into a .env and delete from here

hostname = 'a2plcpnl0542.prod.iad2.secureserver.net'
port = 3306
database_name = 'LambdaSchool'
username = 'jmor'
password = 'zYy0^i@xB-Lm'

In [1]:
pip install mysql-connector

  Stored in directory: C:\Users\John\AppData\Local\pip\Cache\wheels\8c\83\a1\f8b6d4bb1bd6208bbde1608bbfa7557504bed9eaf2ecf8c175
Successfully built mysql-connector
Note: you may need to restart the kernel to use updated packages.


In [4]:
mydb = mysql.connector.connect(
  host=hostname,
  user=username,
  passwd=password,
  db=database_name
)

cursor = mydb.cursor(prepared=True)

In [5]:
csv_data = pd.read_csv('clean_data_no_dup.csv')

In [6]:
print(csv_data.shape)
columns = csv_data.columns.to_list()
columns

(649853, 17)


['id',
 'name',
 'blurb',
 'category',
 'campaign_duration_days',
 'goal_usd',
 'country',
 'city',
 'state',
 'date_created',
 'deadline_date',
 'launched_date',
 'state_changed_at',
 'backers_count',
 'usd_pledged',
 'url',
 'target']

In [7]:
create_clean_data_2 = """
    CREATE TABLE clean_data_2 (
        id INT,
        campaignName VARCHAR(100),
        description VARCHAR(200),
        categories VARCHAR(50),
        duration INT,
        monetaryGoal INT,
        country VARCHAR(50),
        city VARCHAR(50),
        state VARCHAR(50),
        date_created DATE,
        deadline_date DATE,
        launched_date DATE,
        state_changed_at DATE,
        backers_count INT,
        usd_pledged INT,
        url VARCHAR(100),
        target VARCHAR(50)
    );
"""



cursor.execute(create_clean_data_2)

In [75]:
# mydb = mysql.connector.connect(
#   host=hostname,
#   user=username,
#   passwd=password,
#   db=database_name
# )
# cursor = mydb.cursor(prepared=True)

# query = ''
# for index, row in csv_data.iterrows():
#         temp = row.tolist()
#         insert_stmt = (
#             "INSERT INTO clean_data (id, campaignName, description, categories, duration, monetaryGoal, country, city, state, date_created, deadline_date, launched_date, state_changed_at, backers_count, usd_pledged, url, target)"
#             "VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
#         )
#         cursor.execute(insert_stmt, temp)


# mydb.commit();

KeyboardInterrupt: 

In [19]:
# Query function
def get_query(query, info_str, curs):
    """
    Execute and display queries
    Add a comment as to what the query is
    Add a line after
    """
    curs.execute(query)
    print(info_str, curs.fetchall(), '\n')

In [23]:
# Number of campaigns successful for the Art category

campaigns_success_category = """
    SELECT COUNT(campaignName), monetaryGoal, usd_pledged FROM clean_data WHERE target = "successful" AND categories = "Art";
"""

get_query(campaigns_success_category, "Art", cursor)

Art [(3412, 635, 652)] 



In [27]:
# {#}% of campaigns in {persons category} are successful
goal_2 = f'SELECT COUNT(*) / (SELECT COUNT(*) FROM clean_data WHERE categories = "Video Art") FROM clean_data WHERE target = "successful" AND categories = "Video Art";'

get_query(goal_2, "Percent successful in Art", cursor)

Percent successful in Art [(Decimal('0.3288'),)] 



In [47]:
goal_3 = f'SELECT AVG(usd_pledged) FROM clean_data WHERE target = "successful" AND categories = "Video Art";'
    
get_query(goal_3, "AVG raised in category", cursor)

AVG raised in category [(Decimal('5508.4167'),)] 



In [61]:
subquery = 'SELECT AVG(usd_pledged - monetaryGoal) AS diff FROM clean_data WHERE target = "successful" AND categories = "Video Art";'
goal_6 = f'SELECT AVG(diff) FROM (SELECT (usd_pledged - monetaryGoal) AS diff FROM clean_data WHERE target = "successful" AND categories = "Video Art");'

get_query(subquery, 'AVG money raised above goal in category', cursor)

AVG money raised above goal in category [(Decimal('898.6389'),)] 



In [206]:
### Reset connection

mydb = mysql.connector.connect(
  host=hostname,
  user=username,
  passwd=password,
  db=database_name
)
cursor = mydb.cursor(buffered=True)

In [195]:
def get_query(query, curs):
    curs.execute(query)
    return(curs.fetchall()[0][0])

def custom_stats(category, goal):
    # Create queries for each data goal

    goal_1 = f'SELECT COUNT(campaignName) FROM clean_data WHERE target = "successful" AND monetaryGoal > {goal};'

    goal_2 = f'SELECT COUNT(campaignName) / (SELECT COUNT(*) FROM clean_data WHERE categories = "{category}" ) FROM clean_data WHERE target = "successful" AND categories = "{category}";'

    goal_3 = f'SELECT AVG(usd_pledged) FROM clean_data WHERE target = "successful" AND categories = "{category}";'
    
    goal_4 = f'SELECT AVG(duration) FROM clean_data WHERE target = "successful" AND monetaryGoal > {goal};'

    # Change goal to estimate or range
    goal_5 = f'SELECT AVG(backers_count) FROM clean_data WHERE target = "successful" AND monetaryGoal = {goal};'

    # Similar to number 3, consider revising 3 to another stat
    goal_6 = f'SELECT AVG(usd_pledged - monetaryGoal) AS diff FROM clean_data WHERE target = "successful" AND categories = "{category}";'

    queries = [goal_1, goal_2, goal_3, goal_4, goal_5, goal_6]

    results = []
    for query in queries:
        x = get_query(query, cursor)
        results.append(float(x))
    
    return results
    print(results)


custom_results = custom_stats("Video Art", 5000)

custom_results[5]

898.6389

In [210]:

# how many campaigns total?
stat1 = 'SELECT COUNT(*) FROM clean_data'

# what percent of kickstarter campaigns are successful?
stat2 = 'SELECT COUNT(*) / (SELECT COUNT(*) FROM clean_data) FROM clean_data WHERE target = "successful";'

# what are the top 5 categories on kickstarter?
stat3 = 'SELECT TOP 5 COUNT(DISTINCT(categories)) FROM clean_data;'

# successful campaigns have an average duration of x days
stat4 = 'SELECT AVG(duration) FROM clean_data WHERE target = "successful";'

# the average amount raised by successful campaigns is $$$$
stat5 = 'SELECT AVG(usd_pledged) FROM clean_data WHERE target = "successful"'

# the category with the highest average amount raised successfully is?


# successful campaigns raised an average of $$$$ more than their initial goal?
stat7 = 'SELECT AVG(usd_pledged - monetaryGoal) AS diff FROM clean_data WHERE target = "successful";'

# successful campaigns had an average of ### backers?
stat8 = 'SELECT AVG(backers_count) FROM clean_data WHERE target = "successful";'

general = [stat1, stat2, stat4, stat5, stat7, stat8]
results = []
for query in general:
    x = get_query(query, cursor)
    results.append(x)
    
results

[289551,
 Decimal('0.5339'),
 Decimal('31.7519'),
 Decimal('25025.3995'),
 Decimal('15517.6138'),
 Decimal('289.6144')]